In [1]:
import os
from glob import glob
import json
import shutil

# Копирование изображений по аннотациям

In [8]:
import os, json, shutil

ds_type = 'val'
ann = json.load(open(f'{ds_type}/{ds_type}.json'))

ann_file_names = set([os.path.basename(name['file_name']) for name in ann['images']])
for ann_img_name in ann_file_names:
    shutil.copy(f'images/{ann_img_name}', f"{ds_type}/images/{ann_img_name}")

# Просмотр ds в COCO (fiftyOne)

In [ ]:
import os

import fiftyone as fo

# Load COCO formatted dataset
coco_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path='datasets/water/test/aug_images',
    labels_path="datasets/water/test/test_aug.json",
    include_id=True,
)

# Verify that the class list for our dataset was imported
print(coco_dataset.default_classes)  # ['airplane', 'apple', ...]

print(coco_dataset)

session = fo.launch_app(coco_dataset)

# webp to jpg

In [18]:
import os
from glob import glob

from PIL import Image
import imageio.v2 as imageio

img_paths = glob('/home/radmin/Desktop/datasets/corrosion/images/*')
for img_path in img_paths:
    filename, file_extension = os.path.splitext(img_path)
    if file_extension == '.webp':
        img = Image.open(img_path).convert('RGB')
        img.save(filename+'.jpg', 'jpeg')
        print(filename)
        os.remove(img_path)

/home/radmin/Desktop/datasets/corrosion/images/87bcb499-354.oil-and-gas-pipelelectric-gas-welding-butt-weld-of-welding-of-a-p_KRzQZam
/home/radmin/Desktop/datasets/corrosion/images/2b99e164-137.rusty-pipes-closeup-1024x683.jpg
/home/radmin/Desktop/datasets/corrosion/images/1c5c78b7-120.wr-960
/home/radmin/Desktop/datasets/corrosion/images/b70b6c75-87.galvanic-corrosion.ashx
/home/radmin/Desktop/datasets/corrosion/images/8dca08fb-75.66578094-tubos-de-acero-para-el-suministro-de-agua-a-la-corrosic3b_4hKHfcV
/home/radmin/Desktop/datasets/corrosion/images/8bac1038-rust-and-damage-to-the-car-body-metal-corrosion-paint-decay-holes-in-_SFTSfT8
/home/radmin/Desktop/datasets/corrosion/images/af38d70f-297.old-oil-pipe-line-corrosion-metal-lying-in-side-pipe-the-asphalt__dekI8o8
/home/radmin/Desktop/datasets/corrosion/images/feaf6f90-56.wr-960
/home/radmin/Desktop/datasets/corrosion/images/36f42ba5-239.3503836f6bd0478995bf4a49dfffd47e
/home/radmin/Desktop/datasets/corrosion/images/7b631f99-251.co

In [15]:
import os
from glob import glob
import json

from PIL import Image
import imageio.v2 as imageio

img_paths = glob('/home/radmin/Desktop/datasets/corrosion/images/*')
result_ann = json.load(open('/home/radmin/Desktop/datasets/corrosion/result.json'))

count = 0
for img_path in img_paths:
    filename, file_extension = os.path.splitext(img_path)
    
    try:
        count += 1
        imageio.imread(img_path)
    except:
        for result_ann_image in result_ann['images']:
            file_name = result_ann_image['file_name']
            print(file_name, filename)
            break
        #     if result_ann_copy_image[file]
        # # img = Image.open(img_path).convert('RGB')
        # # img.save(filename+'.jpg', 'jpeg')
        # print(filename)
        # # os.remove(img_path)
        break
        

ff8f6ef8-346.depositphotos_234110916-stock-photo-odessa-ukraine-circa-2017-repair.jpg


In [19]:
len(glob('/home/radmin/Desktop/datasets/corrosion/images/*'))
# len(img_paths)

1137

In [22]:
import imageio.v2 as imageio
from glob import glob

ds_type = 'test'
path_to_annotation = f'{ds_type}/{ds_type}.json'
path_to_images = f'{ds_type}/images'
count = 0
for img_path in glob(f"/home/radmin/Desktop/datasets/corrosion/images/*"):
    try:
        imageio.imread(img_path)
    except:
        os.remove(img_path)
        print(img_path)
        count += 1
        
count

0

# Аугментация

In [3]:
from glob import glob
import os
import json

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import imageio.v2 as imageio
import imgaug as ia
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from imgaug import augmenters as iaa 
%matplotlib inline
ia.seed(1)

In [4]:
seq = iaa.Sequential(
    [
        iaa.Sometimes(
            0.5,
            iaa.Affine(scale=(1, 1.5))
        ),
        iaa.Sometimes(
            0.3,
            iaa.Affine(rotate=(-45, 45), mode='edge')
        ),
        iaa.OneOf([
            iaa.SigmoidContrast(gain=10, cutoff=0.5),
            iaa.Snowflakes(flake_size=(0.1, 0.3), speed=(0.01, 0.04)),
            iaa.Fliplr(p=1.0),
            iaa.imgcorruptlike.Spatter(severity=2),
            iaa.Flipud(p=1.0),
            iaa.SigmoidContrast(cutoff=0.5),
            iaa.AllChannelsCLAHE(clip_limit=5),
            iaa.imgcorruptlike.Snow(severity=2),
            iaa.imgcorruptlike.ZoomBlur(severity=2),
            iaa.MaxPooling(1),
            iaa.ScaleX((1.5, 2))
        ])
    ],
    random_order=True
) # apply augmenters in random order

In [7]:
#TODO: распараллелить с помощью parallelbar.progress_map (https://stackoverflow.com/questions/66372005/showing-tqdm-progress-bar-while-using-python-multiprocessing)
ds_type = 'train'
path_to_annotation = f'corrosion/{ds_type}/{ds_type}.json'
path_to_images = f'corrosion/{ds_type}/images'

for img_path in glob(f"{path_to_images}/*"):
    imageio.imread(img_path)

ann_json = json.load(open(path_to_annotation))

aug_ann = dict()
aug_ann['categories'] = ann_json['categories']
aug_ann['info'] = ann_json['info']
aug_ann['images'] = []
aug_ann['annotations'] = []

NUMBER_OF_AUG_IMGS = 10

img_id = 0
ann_id = 0
with tqdm(total=len(ann_json['images'])) as progress:
    for ann_json_image in ann_json['images']:
        images = np.array(
            [imageio.imread(f"{path_to_images}/{ann_json_image['file_name']}", pilmode='RGB') for _ in range(NUMBER_OF_AUG_IMGS)],
            dtype=np.uint8
        )
        
        bbs = []
        for ann_json_ann in ann_json['annotations']:
            if ann_json_ann['image_id'] == ann_json_image['id']:
                bbox = BoundingBox(
                    x1=ann_json_ann['bbox'][0],
                    x2=ann_json_ann['bbox'][0] + ann_json_ann['bbox'][2],
                    y1=ann_json_ann['bbox'][1],
                    y2=ann_json_ann['bbox'][1] + ann_json_ann['bbox'][3],
                    label=ann_json_ann['category_id']
                )
                
                bbs.append(bbox)
        
        bbs = BoundingBoxesOnImage(bbs, shape=images[0].shape)
        bboxes = [bbs for _ in range(NUMBER_OF_AUG_IMGS)]
        
        images_aug, bboxes_aug = seq(images=images, bounding_boxes=bboxes)
        
        bboxes_aug_without_leaving_image = []
        for el in bboxes_aug:
            bboxes_aug_without_leaving_image.append(el.clip_out_of_image())
        
        for img_aug, bbox_aug in zip(images_aug, bboxes_aug_without_leaving_image):

            name, ex = os.path.splitext(ann_json_image['file_name'])
            file_name = f"{name}_{img_id}{ex}"

            imageio.imsave(f"corrosion/{ds_type}/aug_images/{file_name}", img_aug) 
            h, w, _ = img_aug.shape

            aug_ann['images'].append(
                {
                    "width": w,
                    "height": h,
                    "id": img_id,
                    "file_name": file_name
                }
            )
            
            for bb in bbox_aug:
                x1, x2, y1, y2 = (int(bb.x1), int(bb.x2), int(bb.y1), int(bb.y2))
                aug_ann['annotations'].append(
                    {
                        "id": ann_id,
                        "image_id": img_id,
                        "category_id": bb.label,
                        "segmentation": [],
                        "bbox": [
                            x1,
                            y1,
                            x2 - x1,
                            y2 - y1
                        ],
                        "ignore": 0,
                        "iscrowd": 0,
                        "area": (x2 - x1) * (y2 - y1)
                    }
                )
                ann_id += 1
            
            img_id += 1
        progress.update(1)
        


with open(f"corrosion/{ds_type}/{ds_type}_aug.json", 'w') as f:
    json.dump(aug_ann, f)

100%|██████████| 916/916 [45:54<00:00,  3.01s/it]  
